# Import Packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd

# Load Config File

In [2]:
def load_config():
    modules = []
    with open('yolov3.cfg', 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
        lines = [line.rstrip().lstrip() for line in lines if line != '' if line[0] != '#']

        module = {}
        for line in lines:
            if line[0]=='[':
                if len(module) > 0:
                    modules.append(module)
                module = {}
                module['type'] = line[1:-1]
            else:
                key, value = line.split('=')
                module[key.strip()] = value.strip()
        modules.append(module)
    return modules

modules = load_config()

# Create Model

## Define model

In [86]:
THRESH = 0.7
BATCH_SIZE = 1
WIDTH = 608
HEIGHT = 608
NUM_BOX = 3

class ConvLayer(nn.Module):
    def __init__(self, idx, in_f, module):
        super(ConvLayer, self).__init__()
        self.create_module(idx, in_f, module)
    
    def create_module(self, idx, in_f, module):
        self.out_f = int(module['filters'])
        kernel_size = int(module['size'])
        stride = int(module['stride'])
        padding = (kernel_size - 1) // 2
        self.add_module('conv', 
                        nn.Conv2d(in_f, 
                                  self.out_f, 
                                  kernel_size, 
                                  stride, padding, 
                                  bias=False if 'batch_normalize' in module else True))
        if 'batch_normalize' in module:
            self.add_module('norm', nn.BatchNorm2d(self.out_f))
        if module['activation'] == 'leaky':
            self.add_module('leaky', nn.LeakyReLU(inplace=True))
    
    def get_out_features(self):
        return self.out_f
        
    def forward(self, x):
        for sub_module in self.children():
            x = sub_module(x)
        return x

class ShortcutLayer(nn.Module):
    def __init__(self, pos):
        super(ShortcutLayer, self).__init__()
        self.pos = pos
    def forward(self, x, y):
        return x+y

class RouteLayer(nn.Module):
    def __init__(self, pos):
        super(RouteLayer, self).__init__()
        self.pos = [int(i) for i in pos.split(',')]
    def forward(self, x):
        return torch.cat(x, 1)

class UpsampleLayer(nn.Module):
    def __init__(self, scale):
        super(UpsampleLayer, self).__init__()
        self.up = nn.Upsample(scale_factor=scale)
    def forward(self, x):
        return self.up(x)

class YOLOLayer(nn.Module):
    def __init__(self, anchors=None):
        super(YOLOLayer, self).__init__()
        self.anchors = anchors
    
    def get_boxes(self, inputs=None):
        print(inputs.shape)
        grid_x = (torch.arange(self.input_w)
                 .repeat(self.input_w,1)
                 .view(1, self.input_w, self.input_w, 1)
                 .repeat(BATCH_SIZE, 1, 1, NUM_BOX))
        grid_y = (torch.arange(self.input_h)
                 .repeat(self.input_h,1)
                 .t()
                 .view(1, self.input_h, self.input_h, 1)
                 .repeat(BATCH_SIZE, 1, 1, NUM_BOX))
        anchor_w = torch.tensor([w/self.stride for w, _ in self.anchors])
        anchor_h = torch.tensor([h/self.stride for _, h in self.anchors])
        
        #Calculate bx, by, bw, bh
        inputs[..., 0] += grid_x
        inputs[..., 1] += grid_y
        inputs[..., 2] = torch.exp(inputs[..., 2]) * anchor_w
        inputs[..., 3] = torch.exp(inputs[..., 3]) * anchor_h
        
        #Truth ground boxes
        inputs[..., :4] *= self.stride
        inputs = inputs.view(BATCH_SIZE, -1, 85)
        print(inputs.shape)
        
        return inputs
        
    def forward(self, inputs=None, targets=None):
        self.input_w = inputs.shape[-2]
        self.input_h = inputs.shape[-1]
        inputs = inputs.view(BATCH_SIZE, NUM_BOX, -1, self.input_w, self.input_h).permute(0,3,4,1,2).contiguous()
        self.stride = WIDTH / self.input_w
        
        #Sigmoid x, y, po and pc
        inputs[..., :2] = torch.sigmoid(inputs[..., :2])
        inputs[..., 4:] = torch.sigmoid(inputs[..., 4:])
        
        return self.get_boxes(inputs=inputs)

    
    
    
class YOLO(nn.Module):
    def __init__(self, modules):
        super(YOLO, self).__init__()
        self.layers = nn.ModuleList()
        self.create_modules(modules)
    
    def create_modules(self, modules):
        out_fs = [3]
        for idx, module in enumerate(modules[1:]):
            if module['type'] == 'convolutional':
                in_f = out_fs[-1]
                t = ConvLayer(idx, in_f, module)
                f = t.get_out_features()
                self.layers.append(t)
            elif module['type'] == 'shortcut':
                pos = int(module['from'])
                t = ShortcutLayer(pos)
                f = out_fs[-1]
                self.layers.append(t)
            elif module['type'] == 'route':
                pos = module['layers']
                t = RouteLayer(pos)
                f = sum([out_fs[i] for i in t.pos])
                self.layers.append(t)
            elif module['type'] == 'upsample':
                scale = module['stride']
                t = UpsampleLayer(scale)
                f = out_fs[-1]
                self.layers.append(t)
            elif module['type'] == 'yolo':
                mask = [int(i) for i in module['mask'].split(',')]
                anchors = [int(value) for value in module['anchors'].split(',')]
                anchors = [(anchors[2*i], anchors[2*i+1]) for i in mask]
                t = YOLOLayer(anchors=anchors)
                f = out_fs[-1]
                self.layers.append(t)
            out_fs.append(f)
            
    def load_weights(self, weight_path=None):
        with open(weight_path, 'rb') as f:
            header = np.fromfile(f, dtype = np.int32, count = 5)
            weights = np.fromfile(f, dtype = np.float32)
            
        idx_w = 0
        for idx, layer in enumerate(self.layers):
            if isinstance(layer, ConvLayer):
                conv_layer = layer.conv
                
                #Load weights to batch norm or conv bias
                if 'batch_normalize' in modules[idx+1]:
                    bn_layer = layer.norm
                    length = bn_layer.bias.numel()
                    for i in ['bias', 'weight', 'running_mean', 'running_var']:
                        x = getattr(bn_layer, i)
                        weight_to_load = torch.from_numpy(weights[idx_w: idx_w+length])
                        weight_to_load = weight_to_load.view_as(x.data)
                        x.data.copy_(weight_to_load)
                        idx_w += length
                else:
                    length = conv_layer.bias.numel()
                    weight_to_load = torch.from_numpy(weights[idx_w: idx_w+length])
                    weight_to_load = weight_to_load.view_as(layer.conv.bias.data)
                    conv_layer.bias.data.copy_(weight_to_load)
                    idx_w += length

                #Load to conv weight
                length = conv_layer.weight.numel()
                weight_to_load = torch.from_numpy(weights[idx_w: idx_w+length])
                weight_to_load = weight_to_load.view_as(conv_layer.weight.data)
                conv_layer.weight.data.copy_(weight_to_load)
                idx_w += length

                print('Loaded to Conv #{}, weight index is {}'.format(idx, idx_w))

    def create_bouding_boxes(self, inputs=None):
        boxes = []
        for i in range(inputs.shape[1]):
            obj_score = inputs[0,i,4]
            
            if obj_score <= THRESH: 
                continue 
            

    def forward(self, x):
        outputs = []
        yolo_outputs = []
        for idx, layer in enumerate(model.layers):
            if isinstance(layer, ConvLayer):
                x = layer(x)
            elif isinstance(layer, ShortcutLayer):
                x = layer(x, outputs[layer.pos])
            elif isinstance(layer, RouteLayer):
                temp = [outputs[i] for i in layer.pos]
                x = layer(temp)
            elif isinstance(layer, UpsampleLayer):
                x = layer(x)
            elif isinstance(layer, YOLOLayer):
                yolo_output = layer(inputs=x)
                yolo_outputs.append(yolo_output)
                x = outputs[-1]
            outputs.append(x)
            print(idx, x.shape)
        yolo_outputs = torch.cat(yolo_outputs, 1)
        
        self.create_bouding_boxes(inputs=yolo_outputs)
        
        return yolo_outputs

## Create model & load weights

In [96]:
model = YOLO(modules)
model

YOLO(
  (layers): ModuleList(
    (0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (1): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (2): ConvLayer(
      (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky): LeakyReLU(negative_slope=0.01, inplace=True)
    )
    (3): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affi

In [88]:
model.load_weights('./weights/yolov3.weights')

Loaded to Conv #0, weight index is 992
Loaded to Conv #1, weight index is 19680
Loaded to Conv #2, weight index is 21856
Loaded to Conv #3, weight index is 40544
Loaded to Conv #5, weight index is 114784
Loaded to Conv #6, weight index is 123232
Loaded to Conv #7, weight index is 197472
Loaded to Conv #9, weight index is 205920
Loaded to Conv #10, weight index is 280160
Loaded to Conv #12, weight index is 576096
Loaded to Conv #13, weight index is 609376
Loaded to Conv #14, weight index is 905312
Loaded to Conv #16, weight index is 938592
Loaded to Conv #17, weight index is 1234528
Loaded to Conv #19, weight index is 1267808
Loaded to Conv #20, weight index is 1563744
Loaded to Conv #22, weight index is 1597024
Loaded to Conv #23, weight index is 1892960
Loaded to Conv #25, weight index is 1926240
Loaded to Conv #26, weight index is 2222176
Loaded to Conv #28, weight index is 2255456
Loaded to Conv #29, weight index is 2551392
Loaded to Conv #31, weight index is 2584672
Loaded to Conv 

In [6]:
model.eval()

a = torch.zeros((2,3,128,128))
result = model(a)

0 torch.Size([2, 32, 128, 128])
1 torch.Size([2, 64, 64, 64])
2 torch.Size([2, 32, 64, 64])
3 torch.Size([2, 64, 64, 64])
4 torch.Size([2, 64, 64, 64])
5 torch.Size([2, 128, 32, 32])
6 torch.Size([2, 64, 32, 32])
7 torch.Size([2, 128, 32, 32])
8 torch.Size([2, 128, 32, 32])
9 torch.Size([2, 64, 32, 32])
10 torch.Size([2, 128, 32, 32])
11 torch.Size([2, 128, 32, 32])
12 torch.Size([2, 256, 16, 16])
13 torch.Size([2, 128, 16, 16])
14 torch.Size([2, 256, 16, 16])
15 torch.Size([2, 256, 16, 16])
16 torch.Size([2, 128, 16, 16])
17 torch.Size([2, 256, 16, 16])
18 torch.Size([2, 256, 16, 16])
19 torch.Size([2, 128, 16, 16])
20 torch.Size([2, 256, 16, 16])
21 torch.Size([2, 256, 16, 16])
22 torch.Size([2, 128, 16, 16])
23 torch.Size([2, 256, 16, 16])
24 torch.Size([2, 256, 16, 16])
25 torch.Size([2, 128, 16, 16])
26 torch.Size([2, 256, 16, 16])
27 torch.Size([2, 256, 16, 16])
28 torch.Size([2, 128, 16, 16])
29 torch.Size([2, 256, 16, 16])
30 torch.Size([2, 256, 16, 16])
31 torch.Size([2, 128, 

In [98]:
model.layers[0].conv.weight.numel()

864

In [99]:
import math

math.sqrt(864)

29.393876913398138